# XOR

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import json, matplotlib
s = json.load( open("styles/bmh_matplotlibrc.json") )
matplotlib.rcParams.update(s)
from IPython.core.pylabtools import figsize
figsize(11, 5)
colores = ["#348ABD", "#A60628","#06A628"]

In [2]:
from mpl_toolkits.mplot3d import Axes3D

In [3]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display

In [4]:
import numpy as np

In [5]:
import pandas as pd

## Función de activación

In [6]:
def logistica(z):
    """
    Devuelve la función logística evaluada componente por componente
    """
    return 1 / (1 + np.exp(-z))

In [7]:
from matplotlib.pyplot import figure

@interact
def plot_log():
    z = np.arange(-5,5,0.1)
    figure(figsize=(4,2))
    plt.plot(z, logistica(z))
    plt.xlabel("$z$")
    plt.ylabel("$\sigma$")
    plt.title("$\sigma = \\frac{1}{1 + e^{-z}}$")

interactive(children=(Output(),), _dom_classes=('widget-interact',))

In [8]:
def derivada_logistica(z):
    """
    Función que, dado un arreglo de valores z
    calcula el valor de la derivada para cada entrada.
    """
    g = logistica(z)
    return g * (1 - g)

In [8]:
@interact
def plot_logp():
    z = np.arange(-5,5,0.1)
    figure(figsize=(4,2))
    plt.plot(z, derivada_logistica(z))
    plt.xlabel("$z$")
    plt.ylabel("$\sigma'$")
    plt.title("$\sigma' = \sigma (1-\sigma)$")

interactive(children=(Output(),), _dom_classes=('widget-interact',))

In [9]:
def derivada_logistica_atajo(val_sigma):
    """
    Función que, dado un arreglo de valores de sigma(z)
    calcula el valor de la derivada para cada entrada.
    
    Si ya se cuenta con esos valores, es más eficiente
    calcular esto directamente.
    """
    logistics = np.vectorize(logistica)
    return logistics(val_sigma)*(1-logistics(val_sigma))
    ## TODO

In [10]:
@interact
def plot_logpa():
    z = np.arange(-5,5,0.1)
    val = logistica(z)
    figure(figsize=(4,2))
    plt.plot(z, derivada_logistica_atajo(val))
    plt.xlabel("$z$")
    plt.ylabel("$\sigma'$")
    plt.title("$\sigma' = \sigma (1-\sigma)$")

interactive(children=(Output(),), _dom_classes=('widget-interact',))

In [11]:
np.random.seed(10)

## Función de error: Entropía cruzada

La función de error más sencilla utilizada para problemas de clasificación es la **entropía cruzada**
\begin{align}
  J(\Theta) =& - \frac{1}{m} \left[ \sum_{i=1}^m \sum_{k=1}^K   y_k^{(i)} \log(h_\Theta(x^{(i)}))_k  + (1 - y_k^{(i)}) \log(1 - h_\Theta(x^{(i)}))_k   \right]    +  \\
  & \frac{\lambda}{2m} \sum_{l=1}^{L-1} \sum_{i=1}^{s_L} \sum_{j=1}^{s_{l+1}} (\theta_{ji}^{(l)})^2
\end{align}
donde:
  * $K$ es el número de neuronas de salida.
  * $s_l$ es el número de neuronas en la capa $l$.
  * $m$ es el número de ejemplares de entrenamiento.
  
En su forma vectorizada, las componentes del gradiente están dadas por:
\begin{align}
  \delta^{(L)} &= (A^{(L)})^T - Y  \\
  \delta^{(L-1)} &= \delta^{(L)} \Theta_{[:,1:]}^{(L-1)} \circ g'(z^{(L-1)}) \\
  ... \\
  \delta^{(1)} &= \delta^{(0)} \Theta_{[:,1:]}^{(1)} \circ g'(z^{(1)}) \\
\end{align}
con:
\begin{align}
  g'(z^{(l)}) = A^{(l)} \circ (1 - A^{(l)})
\end{align}
en general:
\begin{align}
  \Delta^{(l)} =& (\delta^{(l+1)})^T A^{(l)}   &   \nabla^{(l)} =& \frac{1}{m}\Delta^{(l)}
\end{align}

In [12]:
def cross_entropy(y, h):
    return - y * np.log(h) - (1 - y) * np.log(1 - h)

In [13]:
@interact
def plot_crossentropy():
    """
    Muestra como, en el rango donde está definida, la entropía cruzada tiene
    sus valores más pequeños donde 'y' y 'h' coinciden, y su valor más alto
    donde éstos son opuestos.
    """
    fig = plt.figure()
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')
    #ax = fig.gca(projection='3d') modified 
    # Datos
    X = np.arange(0.01, 1, 0.05)
    Y = np.arange(0.01, 1, 0.05)
    X, Y = np.meshgrid(X, Y)
    CE = cross_entropy(X, Y)
    surf = ax.plot_surface(X, Y, CE, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)
    ax.set_xlabel("$y$")
    ax.set_ylabel("$h$")
    ax.set_title("Entropía cruzada")

interactive(children=(Output(),), _dom_classes=('widget-interact',))

## Red neuronal
La red implementa encadenamiento hacia adelante (para evaluar) y hacia atrás (para entrenarse).

In [14]:
# Cuando se programa un algoritmo que depende de la generación de números aleatorios
# es buena costumbre fijar la semilla a partir de la cual dichos números son generados.
# De este modo el comportamiento sobre los datos de entrada será predecible mientras
# se está desarrollando/probando el código.
#
# Una vez el código está listo, se debe usar variando los valores de esta semilla
# para ver los efectos de la aleatoriedad simulada.

np.random.seed(10)

In [17]:
class Multicapa:
    """
    Red neuronal con tres capas:
    
    Entrada
    Oculta
    Salida
    
    Los parámetros (pesos) que conectan las capas se encuentran en matrices
    con los nombres siguientes:
    
    Entrada -> self.Theta_0 -> Oculta
    Oculta  -> self.Theta_1 -> Salida
    """
    
    def __init__(self, n_entrada, n_ocultas, n_salidas):
        """
        Inicializa la red neuronal, con pesos Theta_0 y Theta_1 aleatorios,
        esta implementación debe incluir el uso de sesgos, por lo que éstos
        no se cuentan en los parámetros siguientes, puedes incluirlos como
        neuronas extra o en sus propias matrices, sólo sé consistente pues
        esto afectará tu implementación.
        
        :param n_entrada: número de datos de entrada (sin contar el sesgo)
        :param n_ocultas: número de neuronas ocultas
        :param n_salidas: número de nueronas de salida
        """
        ## TODO
        self.n_entrada=n_entrada
        self.n_ocultas=n_ocultas
        self.n_salidas=n_salidas

        ##Pesos aleatorios
        self.Theta_0=np.random.randn(self.n_ocultas,self.n_entrada)
        self.Theta_1=np.random.randn(self.n_ocultas,self.n_salidas)
        self.error=0
        self.Grad_0=np.zeros((self.n_entrada,self.n_ocultas))
        self.Grad_1=np.zeros((self.n_ocultas,self.n_salidas))


    def feed_forward(self, X, vector = None):
        """ Calcula las salidas, dados los datos de entrada en forma de matriz.
        Guarda los parámetros siguientes:
        A0: activaciones de la capa de entrada, ya con sesgos
        Z1: potenciales de la capa oculta, aún sin sesgo
        A1: activaciones de la capa oculta, ya con sesgos
        Z2: potencales de la capa de salida
        A2: activaciones de la capa de salida
        
        :param vector: [opcional] se utilizarán los pesos indicados en este
                       vector en lugar de los pesos actuales de la red.
        """
        #Colocamos 1's en las primeras columnas
        A0 = np.insert(X, 0, 1, axis=1)
        i,j=A0.shape
        if vector is None:
            # Obtenemos los pesos actuales de la red
            Theta_0 = self.Theta_0
            Theta_1 = self.Theta_1
        else:
            # Obtenemos los pesos del vector dado
            Theta_0,Theta_1 = self.reconstruct_matrices(vector)
            print("there's vector")

            # Calculamos la capa oculta
            
            
            
            
        self.Theta_0=np.random.randn(j,self.n_ocultas)
        """
        @Daniel
        
        En esta parte creo no se debe ejecutar siempre la linea anterior
        dado que al actualizar las thetas, las volvemos a crear cada
        que se le da una entrada a la red.
        """
        
        
        Theta_0=self.Theta_0
        print("Theta0-2", Theta_0)
        Z1 = np.dot(A0, Theta_0)
        A1 = logistica(Z1)
        A1 = np.insert(A1, 0, 1, axis=1)  # Agregamos el sesgo a la capa oculta
        #Agrega bias a theta1
        Theta_1=np.random.randn(A1.shape[1],self.n_salidas)
        #Theta_1=np.insert(Theta_1,0, np.random.randn(), axis=0)#Agregando sesgo
        
        #print("T2",Theta_1)
        #print("A1", A1)
        # Calculamos la capa de salida
        Z2 = np.dot(A1, Theta_1)
        A2 = logistica(Z2)


        # Guardamos los resultados en atributos de la red
        self.A0 = A0
        self.Z1 = Z1
        self.A1 = A1
        self.Z2 = Z2
        self.A2 = A2


    def back_propagate(self, X, Y, lambda_r = 0.0):
        """ Calcula el error y su gradiente dados los pesos actuales de la red
        y los resultados esperados.
        
        Guarda el error en el atributo self.error y el gradiente en matrices
        self.Grad_1 y self.Grad_0, que tienen la misma forma de Theta_0 y Theta_1.
        
        :param X: matriz de entradas
        :param Y: matriz de salidas deseadas
        :param lambda_r: coeficiente de regularización
        """
        ## TODO
           # Forward pass
        self.feed_forward(X)
    
        # Error en la capa de salida
        delta2 = self.A2 - Y
    
        # Potenciales de la capa oculta, incluyendo el sesgo
        Z1_with_bias = np.insert(self.Z1, 0, 1, axis=1)
    
        self.Grad_1 = Z1_with_bias.T @ delta2 + lambda_r * np.insert(self.Theta_1[1:], 0, 0, axis=0)
    
        # Error en la capa oculta
        delta1 = delta2 @ self.Theta_1.T * derivada_logistica_atajo(Z1_with_bias)
        delta1 = delta1[:, 1:]
    
        # Entradas de la red, incluyendo el sesgo
        A0_with_bias = np.insert(self.A0, 0, 1, axis=1)
    
        # Gradiente de Theta_0 con regularización
        self.Grad_0 = A0_with_bias.T @ delta1 + lambda_r * np.insert(self.Theta_0[1:], 0, 0, axis=0)
    
        # Error de la red con regularización
        self.error = (-Y * np.log(self.A2) - (1 - Y) * np.log(1 - self.A2)).mean() \
                 + (lambda_r / 2) * (np.sum(self.Theta_0[1:] ** 2) + np.sum(self.Theta_1[1:] ** 2))

        
    def calc_error(self, X, Y, vector=None, lambda_r = 0.0):
        """
        Calcula el error que se cometería utilizando los pesos en 'vector' en lugar
        de los pesos actuales de la red.
        
        :returns: el error
        """
        ## TODO
        #A0,A1,A2,Z1,Z2=self.feed_forward(X,vector=vector)
        #self.agregar_elemento_a_cada_renglon(Y,1)
        #error=np.sum((A2-Y)**2)/X.shape[1]
        self.feed_forward(X, vector=vector)
        myA2 = np.insert(self.A2, 0, 1, axis=1)
        print("a2 de error",myA2)
        print("original A2",self.A2)
        m=Y.shape[0]
        
        
        
        J=(-1/m)*np.sum(Y*np.log(myA2) + (1-Y)*np.log(1-myA2))
        if lambda_r>0:
            reg_term=(lambda_r / (2*m)) * (np.sum(np.square(self.Theta_0)) + np.sum(np.square(self.Theta_1)))
            J+=reg_term
        return J


    def vector_weights(self):
        """
        Acomoda a todos los parámetros en las matrices de sesgos y pesos, en un solo vector.
        
        :returns: vector de parámetros
        """
        ## TODO
        #print("thetha0", self.Theta_0, self.Theta_0.ravel())
        #print("thetha1", self.Theta_1, self.Theta_1.ravel())
        return np.concatenate((self.Theta_0.ravel(), self.Theta_1.ravel()))
    
    def reconstruct_matrices(self, vector):
        """
        Dado un vector, rearma matrices del tamaño de las matrices de sesgos y pesos.
        
        :returns: matrices de parámetros
        """
        # Reshape and index to get weight matrices
        n_inputs = self.Theta_0.shape[0]
        n_hidden = self.Theta_0.shape[1]
        n_outputs = self.Theta_1.shape[1]
        print()
        T0 = vector[:((n_inputs ) * n_hidden)].reshape(n_inputs, n_hidden)
        T1 = vector[((n_inputs ) * n_hidden):].reshape(n_hidden , n_outputs)
        
        return T0, T1


        
    def approx_gradient(self, X, Y, lambda_r = 0.0):
        """
        Aproxima el valor del gradiente alrededor de los pesos actuales,
        perturbando cada peso, uno por uno hasta estimar la variación alrededor
        de cada peso.
        
        En este método se itera sobre cada peso w:
        * Sea w - epsilon -> val1, se calcula el error e1 cometido por la red si w es
                                   reemplazado por val1.
        * Sea w + epsilon -> val2, se calcula el error e2 cometido por la red si w es
                                   reemplazado por val2.
        * La parcial correspondiente se estima como (val1 - val2)/(2 * epsilon)
        
        Este método sólo se utiliza para verificar que backpropagation esté bien
        implementado, ya que en la práctica es muy lento y menos preciso.
        
        :returns: matrices que tienen la misma forma de Theta_0 y Theta_1, donde
                  cada entrada es la estimación de la parcial del error con
                  respecto al peso correspondiente
        """
        ## TODO
            # Inicializar las matrices de gradiente
        Theta_0_grad = np.zeros(self.Theta_0.shape)
        Theta_1_grad = np.zeros(self.Theta_1.shape)
    
        # Definir el valor de epsilon
        epsilon = 1e-4
    
        # Iterar sobre los pesos de Theta_0
        for i in range(self.Theta_0.shape[0]):
            for j in range(self.Theta_0.shape[1]):
                theta_temp = np.copy(self.Theta_0)
            
                
                theta_temp[i,j] = theta_temp[i,j] - epsilon
                A1, A2 = self.forward_propagation(X, theta_temp, self.Theta_1)
                cost1 = self.calc_error(A2, Y, theta_temp, self.Theta_1, lambda_r)
                theta_temp = np.copy(self.Theta_0)
                theta_temp[i,j] = theta_temp[i,j] + epsilon
                A1, A2 = self.forward_propagation(X, theta_temp, self.Theta_1)
                cost2 = self.calc_error(A2, Y, theta_temp, self.Theta_1, lambda_r)
                Theta_0_grad[i,j] = (cost1 - cost2) / (2 * epsilon)
    
        # Iterar sobre los pesos de Theta_1
        for i in range(self.Theta_1.shape[0]):
            for j in range(self.Theta_1.shape[1]):
                theta_temp = np.copy(self.Theta_1)
                theta_temp[i,j] = theta_temp[i,j] - epsilon
                A1, A2 = self.forward_propagation(X, self.Theta_0, theta_temp)
                cost1 = self.calc_error(A2, Y, self.Theta_0, theta_temp, lambda_r=lambda_r)
                theta_temp = np.copy(self.Theta_1)
                theta_temp[i,j] = theta_temp[i,j] + epsilon
                A1, A2 = self.forward_propagation(X, self.Theta_0, theta_temp)
                cost2 = self.calc_error(A2, Y, self.Theta_0, theta_temp, lambda_r)
                Theta_1_grad[i,j] = (cost1 - cost2) / (2 * epsilon)
        return Theta_0_grad,Theta_1_grad
        
    def gradient_descent(self, X, Y, alpha, ciclos=10, check_gradient = False, lambda_r = 0.0):
        """ Evalúa y ajusta los pesos de la red, de acuerdo a los datos en X y los resultados
        deseados en Y.  Al final grafica el error vs ciclo.  Si el entrenamiento es correcto
        el error debe descender por cada iteración (ciclo).
        
        :param X: datos de entrada
        :param Y: salidas deseadas
        :param alpha: taza de aprendizaje
        :param ciclos: número de veces que se realizarán ajustes para todo el conjunto de datos X
        :param check_gradient: se calculará el gradiente con backpropagation y con aproximación por
                               perturbaciones, imprimiendo los valores lado a lado para que puedan
                               ser comparados.
        :param lambda_r: coeficiente de regularización
        """
        ## TODO
        error_por_ciclo = np.zeros(ciclos)

        for i in range(ciclos):
            self.feed_forward(X)

            # Calculamos el error
            error = self.calc_error(self.A2, Y, lambda_r=lambda_r)

            # Calculamos el gradiente usando backpropagation
            self.back_propagate(X,Y,lambda_r=lambda_r)
            # Si se desea, se calcula el gradiente mediante aproximación por perturbaciones
            if check_gradient:
                approx_grad1, approx_grad2 = self.approx_gradient(X, Y, lambda_r=lambda_r)
                print("Gradiente con backpropagation:\n")
                print(self.D1)
                print(self.D2)
                print("\nGradiente con aproximación por perturbaciones:\n")
                print(approx_grad1)
                print(approx_grad2)
            self.update_weights(self.D1, self.D2,alpha)
            error_por_ciclo[i] = error
        plt.plot(error_por_ciclo)
        plt.title("Error vs ciclo")
        plt.xlabel("Ciclo")
        plt.ylabel("Error")
        plt.show()

    def update_weights(self,d0,d1,alpha):
        """
        Función auxiliar para actualizar los pesos
        """
        self.Theta_0=self.Theta_0-alpha*d0
        self.Theta_1=self.Theta_1-alpha*d1
        
    def print_output(self):
        """
        Muestra en pantalla los valores de salida obtenidos en la última ejecución de feed_forward.
        """
        ## TODO
        print(self.A2)
        
    
    def confusion_matrix(Y):
        pass


# Conjunto de datos

Probaremos esta red con la función más sencilla que requiere más de un perceptrón: $XOR$.

Para ello requerimos una red con la arquitectura siguiente:

<img src="figuras/xor_sin_pesos.png">

In [18]:
# Datos
X = np.array([[0, 0], 
              [0, 1], 
              [1, 0],
              [1, 1]])  # Entradas
Y = np.array([[0], [1], [1], [0]])              # Salidas deseadas

# Instanciación de la red
xor = Multicapa(2, 2, 1)

# Probamos qué valores calcula con una inicialización aleatoria
xor.feed_forward(X)
xor.print_output()

Theta0-2 [[ 0.26551159  0.10854853]
 [ 0.00429143 -0.17460021]
 [ 0.43302619  1.20303737]]
[[0.43473481]
 [0.47546745]
 [0.4325518 ]
 [0.47396381]]


Para verificar si la red funciona para lo que fue diseñada, no basta con reducir el error,
debemos evaluar si calcula la función que queremos que calcule.  En este caso, queremos
usarla para evaluar una función binaria.  Si realizamos un redondeo, veamos cuántas respuestas
calcula correctamente.

In [27]:
def count_correct(red, X, Y):
    """ Conciderando que las etiquetas en Y son binarias (0 y 1)
    la salida h de la red se tomará como 0 si h < 0.5
    y como 1 si h >= 0.5
    """
    red.feed_forward(X)
    H = red.A2                     # Ojo: dependiendo de tu implementación, podrías no querer la .T
    H = np.rint(H)
    print("H = ", H)
    print("Y = ", Y)
    c = np.count_nonzero((H - Y)==0)
    print("Se calcularon correctamente ", c, "entradas.")
    
    return H
    

In [28]:
H = count_correct(xor, X, Y)

Theta0-2 [[-0.47614201  1.30847308]
 [ 0.19501328  0.40020999]
 [-0.33763234  1.25647226]]
H =  [[0.]
 [0.]
 [0.]
 [0.]]
Y =  [[0]
 [1]
 [1]
 [0]]
Se calcularon correctamente  2 entradas.


In [29]:
H

array([[0.],
       [0.],
       [0.],
       [0.]])

Para dar una idea general del comportamiento de la matriz, para ello creamos la matriz de confusion
la cual muestra los errores de tipo uno asi como de tipo dos en el siguiente formato:

In [56]:
def matriz_confusion_xor(H, Y):
    
    """ 
    :param_H = Salida predecida por la red
    :param_Y = Salida esperada 
    """
    
    """"
    Donde las predicciones correctas son las que se encuentran en la 
    diagonal, igualmente con las etuquetas de la tabla, se obtiene que predijo y 
    lo esperado, 
    
    
    m[1][0]: falso positivo -> Error del tipo 1
    m[0][1]: falso negativo -> Error del tipo 2 
    
    dada la compuera XOR, se eligio unicamente hacerlo con una secuencia de 
    condicionales en lugar de ciclos, ya que no tenemos mas opciones de salia.
    
    """
    
    m = [[0,0],
         [0,0]]
    
    for i in range(Y.shape[0]):
        if Y[i] == 0:
            if H[i] == 0:
                m[0][0] += 1
            else:
                m[0][1] += 1
        
        
        else:
            if H[i] == 1:
                m[1][1] += 1
            else:
                m[1][0] += 1
                
        
            
    
    
    return m

In [57]:
pd.DataFrame.from_records(matriz_confusion_xor(H,Y))

,0,1
0,2,0
1,2,0


Para poder implementar el algoritmo de optimización, las variables a optimizar, es decir, los pesos, deben ser colocados sobre un vector (o matriz columna, en este caso).  Es necesario convertir las matrices de pesos a vector y, para usar la red, revertir el proceso, reconstruyendo las matrices a partir del vector con los pesos.

In [69]:
# Con esta casilla verifica que tus conversiones sean correctas

print("Valores actuales de los pesos")

print("Theta_0:\n", xor.Theta_0, "\n\nTheta_1:\n", xor.Theta_1)

print("\nMatrices a vector de pesos: ")
print(xor.vector_weights())

print("\nReconstrucción de las matrices a partir del vector de pesos: ")
T0, T1 = xor.reconstruct_matrices(xor.vector_weights())
print(T0, T1)

Valores actuales de los pesos
Theta_0:
 [[ 0.15701498  1.35878914]
 [ 0.51534525  2.38412462]
 [ 0.52919027 -0.89614384]] 

Theta_1:
 [[ 0.5508447 ]
 [-0.19152205]]

Matrices a vector de pesos: 
[ 0.15701498  1.35878914  0.51534525  2.38412462  0.52919027 -0.89614384
  0.5508447  -0.19152205]

Reconstrucción de las matrices a partir del vector de pesos: 

[[ 0.15701498  1.35878914]
 [ 0.51534525  2.38412462]
 [ 0.52919027 -0.89614384]] [[ 0.5508447 ]
 [-0.19152205]]


In [70]:
# Con esta casilla revisa que backpropagation y aproximación por perturbaciones
# den resultados semejantes.  Observa que se ejecuta sobre un solo ciclo pues es lento.

xor.gradient_descent(X, Y, 0.3, 1, check_gradient = True)


Theta0-2 [[ 0.6767082   0.06072782]
 [-0.02304119 -0.55331843]
 [-1.10612324 -0.21713489]]
Theta0-2 [[-1.41575321  0.09470831]
 [ 0.19380061  0.54323509]]
a2 de error [[1.         0.50027957]
 [1.         0.50020298]
 [1.         0.50007143]
 [1.         0.49999727]]
original A2 [[0.50027957]
 [0.50020298]
 [0.50007143]
 [0.49999727]]
-0.25
Theta0-2 [[ 1.10950523 -0.99263248]
 [-0.68234147 -0.4092692 ]
 [-0.24747527  1.52785112]]


/tmp/ipykernel_4949/842624618.py:151: RuntimeWarning: divide by zero encountered in log
  J=(-1/m)*np.sum(Y*np.log(myA2) + (1-Y)*np.log(1-myA2))
/tmp/ipykernel_4949/842624618.py:151: RuntimeWarning: invalid value encountered in multiply
  J=(-1/m)*np.sum(Y*np.log(myA2) + (1-Y)*np.log(1-myA2))


ValueError: operands could not be broadcast together with shapes (3,1) (2,1) 

In [47]:
@interact_manual(ciclos = (50, 2000))
def train_XOR(ciclos):
    xor.gradient_descent(X, Y, 0.3, ciclos)

interactive(children=(IntSlider(value=1025, description='ciclos', max=2000, min=50), Button(description='Run I…

In [21]:
# Después de haber sido correctamente entrenada, la red debe producir salidas
# cercanas a las deseadas.  Dado que la función de activación es la sigmoide
# nunca llegará a 0 o 1 exactamente.

xor.feed_forward(X)
xor.print_output()

print("\nTheta_0 = ", xor.Theta_0, "\n\nTheta_1", xor.Theta_1)

Theta0-2 [[-0.52929608  1.04618286]
 [-1.41855603 -0.36249918]
 [-0.12190569  0.31935642]]
[[0.75266342]
 [0.76402969]
 [0.74862142]
 [0.76140171]]

Theta_0 =  [[-0.52929608  1.04618286]
 [-1.41855603 -0.36249918]
 [-0.12190569  0.31935642]] 

Theta_1 [[ 0.62133597]
 [-0.72008556]]


In [22]:
c = count_correct(xor, X, Y)


Theta0-2 [[ 0.31475378  2.46765106]
 [-1.50832149  0.62060066]
 [-1.04513254 -0.79800882]]
H =  [[1.]
 [1.]
 [1.]
 [1.]]
Y =  [[0]
 [1]
 [1]
 [0]]
Se calcularon correctamente  2 entradas.


## Regularización

In [23]:
xor_reg = Multicapa(2, 2, 1)

In [24]:
c = count_correct(xor_reg, X, Y)

Theta0-2 [[-1.99439377  1.10770823]
 [ 0.24454398 -0.06191203]
 [-0.75389296  0.71195902]]
H =  [[1.]
 [1.]
 [1.]
 [1.]]
Y =  [[0]
 [1]
 [1]
 [0]]
Se calcularon correctamente  2 entradas.


In [ ]:
xor.gradient_descent(X, Y, 0.3, 1, check_gradient = True, lambda_r = 0.15)

Theta0-2 [[-1.85618548 -0.2227737 ]
 [-0.06584785 -2.13171211]
 [-0.04883051  0.39334122]]
Theta0-2 [[ 0.24454398 -0.06191203]
 [-0.75389296  0.71195902]]
a2 de error [[1.         0.68047041]
 [1.         0.68111298]
 [1.         0.67831673]
 [1.         0.67861215]]
original A2 [[0.68047041]
 [0.68111298]
 [0.67831673]
 [0.67861215]]
Theta0-2 [[ 0.82699862 -1.95451212]
 [ 0.11747566 -1.90745689]
 [-0.92290926  0.46975143]]


/tmp/ipykernel_17092/1317291660.py:146: RuntimeWarning: divide by zero encountered in log
  J=(-1/m)*np.sum(Y*np.log(myA2) + (1-Y)*np.log(1-myA2))
/tmp/ipykernel_17092/1317291660.py:146: RuntimeWarning: invalid value encountered in multiply
  J=(-1/m)*np.sum(Y*np.log(myA2) + (1-Y)*np.log(1-myA2))


ValueError: operands could not be broadcast together with shapes (3,1) (2,1) 

In [ ]:
@interact_manual(ciclos = (50, 2000))
def train_XOR_reg(ciclos):
    # Prueba diferentes valores de lambda_r ¿qué tanto lo puedes incrementar si matar a la red?
    xor_reg.gradient_descent(X, Y, 0.3, ciclos, lambda_r = 0.005)

interactive(children=(IntSlider(value=1025, description='ciclos', max=2000, min=50), Button(description='Run I…

In [ ]:
xor_reg.feed_forward(X)
xor_reg.print_output()
print("Theta_0 = ", xor_reg.Theta_0, "\nTheta_1", xor_reg.Theta_1)

Theta0-2 [[ 0.84820861  0.70683045]
 [-0.78726893  0.29294072]
 [-0.47080725  2.40432561]]
[[0.23289374]
 [0.22115581]
 [0.23947263]
 [0.23115213]]
Theta_0 =  [[ 0.84820861  0.70683045]
 [-0.78726893  0.29294072]
 [-0.47080725  2.40432561]] 
Theta_1 [[ 0.4609029 ]
 [-0.21578989]]


In [ ]:
c = count_correct(xor_reg, X, Y)

Theta0-2 [[-0.43902624  0.14110417]
 [ 0.27304932 -1.61857075]
 [-0.57311336 -1.32044755]]
H =  [[1.]
 [1.]
 [1.]
 [1.]]
Y =  [[0]
 [1]
 [1]
 [0]]
Se calcularon correctamente  2 entradas.


In [ ]:
from IPython.core.display import HTML
def css_styling():
    styles = open("styles/custom.css", "r").read() #or edit path to custom.css
    return HTML(styles)
css_styling()